# CS 525 Assignment 1
Sirut Buasai, sbuasai2@wpi.edu

### Imports and Downloads

In [6]:
# libraries
import matplotlib.pyplot as plt
import nltk
import pandas as pd
from wordcloud import WordCloud

# NLTK downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sirutbuasai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sirutbuasai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sirutbuasai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sirutbuasai/nltk_data...


True

### Data retrieval and processing

In [8]:
# load data from csv files
real_data = pd.read_csv('True.csv')
fake_data = pd.read_csv('Fake.csv')

# clean the data by removing punctuations and special characters and convert string to lower case
real_data = real_data.replace(r'[^A-Za-z0-9]+', ' ', regex=True)
real_data['text'] = real_data['text'].str.lower()

fake_data = fake_data.replace(r'[^A-Za-z0-9]+', ' ', regex=True)
fake_data['text'] = fake_data['text'].str.lower()

## Task 1: Explore Essential Information from Text Data and Preprocessing
### Text Tokenization

In [9]:
# tokenize texts
real_data['tokenized_text'] = real_data['text'].apply(nltk.tokenize.word_tokenize)
real_tokens = real_data['tokenized_text'].explode()
real_tokens.dropna(inplace=True)
real_tokens = real_tokens.to_list()

fake_data['tokenized_text'] = fake_data['text'].apply(nltk.tokenize.word_tokenize)
fake_tokens = fake_data['tokenized_text'].explode()
fake_tokens.dropna(inplace=True)
fake_tokens = fake_tokens.to_list()

collection_tokens = real_tokens + fake_tokens

### Stop Word Removal

In [10]:
# remove stop words
stop_words = nltk.corpus.stopwords.words('english')
real_stop = [x for x in real_tokens if x not in stop_words]
fake_stop = [x for x in fake_tokens if x not in stop_words]
collection_stop = [x for x in collection_tokens if x not in stop_words]

### Text Lemminization

In [11]:
# lemmatize tokens
lemmatizer = nltk.stem.WordNetLemmatizer()
real_lemm = [lemmatizer.lemmatize(x) for x in real_stop]
fake_lemm = [lemmatizer.lemmatize(x) for x in fake_stop]
collection_lemm = [lemmatizer.lemmatize(x) for x in collection_stop]

### Calculate Top 100 Common Words and WordCloud Analysis

In [12]:
# retrieve top 100 most common words
real_freq = nltk.FreqDist(real_lemm).most_common(100)
fake_freq = nltk.FreqDist(fake_lemm).most_common(100)
collection_freq = nltk.FreqDist(collection_lemm).most_common(100)

# download tables to excel spreadsheet
real_df = pd.DataFrame(data=dict(real_freq), index=[0])
real_df = (real_df.T)
real_df.to_excel('real_freq.xlsx')

fake_df = pd.DataFrame(data=dict(fake_freq), index=[0])
fake_df = (fake_df.T)
fake_df.to_excel('fake_freq.xlsx')

collection_df = pd.DataFrame(data=dict(collection_freq), index=[0])
collection_df = (collection_df.T)
collection_df.to_excel('collection_freq.xlsx')

# create wordcloud for analysis
real_wordcloud = WordCloud().generate_from_frequencies(dict(real_freq))
plt.imshow(real_wordcloud)
plt.show()

fake_wordcloud = WordCloud().generate_from_frequencies(dict(fake_freq))
plt.imshow(fake_wordcloud)
plt.show()

collection_wordcloud = WordCloud().generate_from_frequencies(dict(collection_freq))
plt.imshow(collection_wordcloud)
plt.show()

## Task 2: Build Machine Learning Model